## Imports and constants

In [ ]:
import math
import numpy as np
import pandas as pd
import shapely
import matplotlib
from matplotlib import pylab as plt
import geopandas as gpd
import pyproj
import contextily as ctx

from shapely.geometry import Point, LineString, MultiPoint, Polygon, GeometryCollection
from shapely.ops import unary_union
from shapely.wkt import loads as wkt_load

In [ ]:
crs_gps = pyproj.CRS.from_epsg('4326') # WGS84 Merkator (degrees)
crs_web = pyproj.CRS.from_epsg('3857') # Pseudo-Merkator (meters)
crs_est = pyproj.CRS.from_epsg('3301') # Estonian Coordinate System

In [ ]:
# https://epsg.io/transform#s_srs=4326&t_srs=3301 - used this to convert from gps to estonian coordinate system
# needed to invert latitude and longitude
delta = Point(659351.9779390106, 6474942.48407838)
lounakeskus = Point(656661.9914198933, 6471868.239886967)
sirius = Point(659234.2258483924, 6471525.160984464)
ihaste = Point(663231.1552057452, 6471198.855079322)
kvissental = Point(657392.1712729766, 6477063.679848602)

In [ ]:
data = pd.read_csv("data/driveways_cleaned.csv", index_col="global_id")
data["geometry"] = data["geometry"].apply(wkt_load)
gdf_data = gpd.GeoDataFrame(data, geometry="geometry", crs=crs_est)
# Remove 3 instances of MultiLineStrings (they don't allow for coords)
gdf_data = gdf_data[gdf_data.geometry.geom_type == "LineString"]
gdf_data["start_node_x"] = gdf_data.geometry.apply(lambda row: round(row.coords[0][0],3))
gdf_data["start_node_y"] = gdf_data.geometry.apply(lambda row: round(row.coords[0][1],3))
gdf_data["end_node_x"] = gdf_data.geometry.apply(lambda row: round(row.coords[-1][0],3))
gdf_data["end_node_y"] = gdf_data.geometry.apply(lambda row: round(row.coords[-1][1],3))
gdf_data.head()
gdf_data.head()

In [ ]:
from collections import defaultdict
# defaultdict of start nodes that points to a defaultdict of end nodes that points to a tuple that ->
# -> contains the distance of the end node and the global id of the road one should take
graph = defaultdict(lambda: defaultdict( lambda: [math.inf, None]))

for index, row in gdf_data.iterrows():
    # there were 3 lines that were MultiLineString which didn't allow for coords, so I excluded them
    if row.geometry.geom_type != "LineString":
        continue
    start_node = (row.start_node_x, row.start_node_y)
    end_node = (row.end_node_x, row.end_node_y)
    graph[start_node][end_node] = [row.length, index] 
    # if traffic is two-way, need to add an edge from end to beginning
    if row.directionality == 0:
        graph[end_node][start_node] = [row.length, index]

## Helpers

In [3]:
def get_closest_road_idx(point, road_table):
    distances_to_roads = point.distance(road_table["geometry"])
    return distances_to_roads.idxmin()

def get_road_start(road_idx, road_table):
    return (gdf_data.loc[road_idx].start_node_x, gdf_data.loc[road_idx].start_node_y)

def get_road_end(road_idx, road_table):
    return (gdf_data.loc[road_idx].end_node_x, gdf_data.loc[road_idx].end_node_y)

def get_node_idx(node_tuple, nodes):
    for i in range(len(nodes)):
        if nodes[i] == node_tuple:
            return i
    return None

def recreate_path(pn, end):
    prev, road = pn[end]
    roads = [road]
    while pn[prev] != None:
        roads.append(road)
        prev, road = pn[prev]
    roads.reverse()
    return roads

## A-star algorithm

In [ ]:
from queue import PriorityQueue
from collections import defaultdict
from math import sqrt

def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def a_star_search(graph, start):
    frontier = PriorityQueue()
    frontier.put((0, start))
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()[1]

        for next, (distance, road_id) in graph[current].items():
            new_cost = cost_so_far[current] + distance
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put((priority, next))
                came_from[next] = current, road_id

    return cost_so_far, came_from

In [ ]:
delta_road = get_closest_road_idx(delta, gdf_data)
start = get_road_start(delta_road, gdf_data)
lounakeskus_road = get_closest_road_idx(lounakeskus, gdf_data)
end_lounakeskus = get_road_end(lounakeskus_road, gdf_data)
sirius_road = get_closest_road_idx(sirius, gdf_data)
end_sirius = get_road_end(sirius_road, gdf_data)
ihaste_road = get_closest_road_idx(ihaste, gdf_data)
end_ihaste = get_road_end(ihaste_road, gdf_data)
kvissental_road = get_closest_road_idx(kvissental, gdf_data)
end_kvissental = get_road_end(kvissental_road, gdf_data)

In [ ]:
start_dijkstra = perf_counter()
distances, prev_roads = pq_dijkstra(graph, start)
print(f'Time taken: {perf_counter() - start_dijkstra}')
path_lounakeskus = recreate_path(prev_roads, end_lounakeskus)
path_lounakeskus.insert(0, delta_road)
path_sirius = recreate_path(prev_roads, end_sirius)
path_sirius.insert(0, delta_road)
path_ihaste = recreate_path(prev_roads, end_ihaste)
path_ihaste.insert(0, delta_road)
path_kvissental = recreate_path(prev_roads, end_kvissental)
path_kvissental.insert(0, delta_road)